In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
# Experiment configuration
experiment_name = "ekf_debug3"
dataset_args = {
    'use_proprioception': True,
    'use_haptics': True,
    'use_vision': True,
    'vision_interval': 2,
}

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm_notebook

import fannypack
from lib import panda_datasets, panda_baseline_models, panda_baseline_training
from lib.ekf import KalmanFilterNetwork
from lib.ekf_models import PandaEKFDynamicsModel, PandaEKFMeasurementModel
from fannypack import utils
from lib import dpf



In [32]:
print("Creating dataset...")
# dataset_full = panda_datasets.PandaParticleFilterDataset(
#     'data/gentle_push_10.hdf5',
#     subsequence_length=16,
#     **dataset_args)

e2e_trainset = panda_datasets.PandaParticleFilterDataset(
    "data/gentle_push_10.hdf5",
    subsequence_length=16,
    particle_count=1,
    particle_variances=(.0005, .0005),
    **dataset_args
)

dataset_dynamics = panda_datasets.PandaDynamicsDataset(
    'data/gentle_push_10.hdf5',
    subsequence_length=16,
    **dataset_args)

dataset_measurement = panda_datasets.PandaMeasurementDataset(
    'data/gentle_push_10.hdf5',
    subsequence_length=16,
    **dataset_args)

Creating dataset...
Parsed data: 15 active, 0 inactive
Keeping (inactive): 0
Parsed data: 239 active, 0 inactive
Keeping: 0



Loaded 240 points


In [33]:
dataloader = torch.utils.data.DataLoader(
    dataset_dynamics, batch_size=16, shuffle=True, num_workers=2, drop_last=True)



In [43]:
measurement = PandaEKFMeasurementModel()
dynamics = PandaEKFDynamicsModel()
ekf = KalmanFilterNetwork(dynamics, measurement)
print("Creating model...")
buddy = fannypack.utils.Buddy(experiment_name, 
                              ekf, 
                              optimizer_names=["ekf", "ekf_dynamics", "ekf_measurement"], 
                              load_checkpoint=False,
)
# training dynamics model 


Creating model...
[buddy-ekf_debug3] Using device: cuda


In [ ]:
dataloader_measurement = torch.utils.data.DataLoader(
    dataset_measurement, batch_size=16, shuffle=True, num_workers=2, drop_last=True)
for _ in tqdm_notebook(range(1000)):
    for batch_idx, batch in enumerate(tqdm_notebook(dataloader_measurement)):
        noisy_state, observation, _, state  = fannypack.utils.to_device(batch, buddy._device)
#         states = states[:,0,:]
        state_update, R = measurement(observation, noisy_state)
        loss = F.mse_loss(state_update, state)
        buddy.minimize(loss,
                       optimizer_name="ekf_measurement",
                       checkpoint_interval=500)
        buddy.log("measurement_loss", loss)
buddy.save_checkpoint()

In [ ]:
# training dynamics model 

dataloader_dynamics = torch.utils.data.DataLoader(
    dataset_dynamics, batch_size=16, shuffle=True, num_workers=2, drop_last=True)
for _ in tqdm_notebook(range(1000)):
    for batch_idx, batch in enumerate(tqdm_notebook(dataloader_dynamics)):
        prev_state, observation, control, new_state = fannypack.utils.to_device(batch, buddy._device)
#         states = states[:,0,:]
        predicted_states = dynamics(prev_state, control)
        
        loss = F.mse_loss(predicted_states, new_state)
        buddy.minimize(loss,
                       optimizer_name="ekf_dynamics",
                       checkpoint_interval=500)
        buddy.log("dynamics_loss", loss)
buddy.save_checkpoint()

/scr-ssd/miniconda3/envs/filter/lib/python3.6/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


/scr-ssd/miniconda3/envs/filter/lib/python3.6/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


[buddy-ekf_debug3] Saved checkpoint to path: checkpoints/ekf_debug3-0000000000000500.ckpt



[buddy-ekf_debug3] Saved checkpoint to path: checkpoints/ekf_debug3-0000000000001000.ckpt



[buddy-ekf_debug3] Saved checkpoint to path: checkpoints/ekf_debug3-0000000000001500.ckpt



[buddy-ekf_debug3] Saved checkpoint to path: checkpoints/ekf_debug3-0000000000002000.ckpt



[buddy-ekf_debug3] Saved checkpoint to path: checkpoints/ekf_debug3-0000000000002500.ckpt



[buddy-ekf_debug3] Saved checkpoint to path: checkpoints/ekf_debug3-0000000000003000.ckpt



[buddy-ekf_debug3] Saved checkpoint to path: checkpoints/ekf_debug3-0000000000003500.ckpt



In [39]:
print(dataset_full)
print(dataloader_full)

# dataloader_full.get_item()

dataset_full.__getitem__(10)

(tensor([[-0.3310,  0.9663],
         [-0.3730,  0.9841],
         [-0.3501,  0.9573],
         [-0.3249,  0.9756],
         [-0.3481,  0.9925],
         [-0.3084,  0.9719],
         [-0.3790,  0.9505],
         [-0.3510,  0.9139],
         [-0.3377,  0.9423],
         [-0.3318,  0.9862],
         [-0.3819,  0.9629],
         [-0.3445,  0.9475],
         [-0.3703,  0.9600],
         [-0.3060,  0.9747],
         [-0.3412,  0.9164],
         [-0.3341,  0.9380],
         [-0.3528,  0.9630],
         [-0.3251,  0.9473],
         [-0.2779,  0.9782],
         [-0.3126,  0.9533],
         [-0.3105,  0.9638],
         [-0.2900,  0.9661],
         [-0.3406,  0.9450],
         [-0.3384,  0.9558],
         [-0.3237,  0.9531],
         [-0.3332,  0.9988],
         [-0.3497,  0.9658],
         [-0.3476,  0.9440],
         [-0.3505,  0.9305],
         [-0.3442,  0.9969],
         [-0.3578,  0.9748],
         [-0.3286,  0.9666],
         [-0.3177,  0.9186],
         [-0.3045,  0.9362],
         [-0.3

In [ ]:
# training e2e model 

log_interval = 10
dataloader_full = torch.utils.data.DataLoader(
    e2e_trainset, batch_size=16, shuffle=True, num_workers=2, drop_last=True)
# for _ in tqdm_notebook(range(10)):



print("running")
for _ in tqdm_notebook(range(100000)):
    for batch_idx, batch in enumerate(tqdm_notebook(dataloader_full)):
#     for batch_idx, batch in enumerate((dataloader_full)):
        # Transfer to GPU and pull out batch data
        batch_gpu = utils.to_device(batch, buddy._device)
        _, batch_states, batch_obs, batch_controls = batch_gpu
        # N = batch size, M = particle count
        N, timesteps, control_dim = batch_controls.shape
        N, timesteps, state_dim = batch_states.shape
        assert batch_controls.shape == (N, timesteps, control_dim)
        
        state = batch_states[:, 0, :]
        state_sigma = torch.eye(state.shape[-1], device=buddy._device)*0.001
        state_sigma = state_sigma.unsqueeze(0).repeat(N, 1, 1)
            
        # Accumulate losses from each timestep
        losses = []
        for t in range(1, timesteps-1):
            prev_state = state
            prev_state_sigma = state_sigma

            state, state_sigma = ekf.forward(
                prev_state,
                prev_state_sigma,
                utils.DictIterator(batch_obs)[:, t, :],
                batch_controls[:, t, :],
                noisy_dynamics=True
            )
            
            assert state.shape == batch_states[:, t, :].shape
            loss = torch.mean((state - batch_states[:, t, :]) ** 2)
            losses.append(loss)
        buddy.minimize(
            torch.mean(torch.stack(losses)),
            optimizer_name="ekf",
            checkpoint_interval=500)

        if buddy.optimizer_steps % log_interval == 0:
            with buddy.log_scope("ekf"):
                buddy.log("Training loss", loss)

        print("Epoch loss:", np.mean(utils.to_numpy(losses)))
buddy.save_checkpoint()



running


/scr-ssd/miniconda3/envs/filter/lib/python3.6/site-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


/scr-ssd/miniconda3/envs/filter/lib/python3.6/site-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


Epoch loss: 1.910075
Epoch loss: 1.612447
Epoch loss: 2.542368



Epoch loss: 0.98175097
Epoch loss: 1.1761341
Epoch loss: 5.222328



...

Epoch loss: 2.5012913
Epoch loss: 1.397846
Epoch loss: 0.49874383



Epoch loss: 1.059052
Epoch loss: 2.4082484
Epoch loss: 1.8828713



Epoch loss: 1.8790306
Epoch loss: 0.7426184
Epoch loss: 1.4360151



Epoch loss: 1.9854922
Epoch loss: 0.62651235
Epoch loss: 1.7652719



Epoch loss: 1.3710535
Epoch loss: 1.7786758
Epoch loss: 1.9089873



In [71]:
print(next(ekf.parameters()).device)


cuda:0


In [133]:
prev_state = torch.Tensor(state.detach())

state, state_sigma = ekf.forward(
                prev_state,
                prev_sta`te_sigma,
                utils.DictIterator(batch_obs)[:, t, :],
                batch_controls[:, t, :],
                noisy_dynamics=True
            )

forward calls
torch.Size([16, 2, 2])
torch.Size([16, 2, 1])


In [42]:
dataloader_measurement = torch.utils.data.DataLoader(
    dataset_full, batch_size=16, shuffle=True, num_workers=2, drop_last=True)
for batch_idx, batch in enumerate(tqdm_notebook(dataloader_measurement)):
    print(batch_idx)

/scr-ssd/miniconda3/envs/filter/lib/python3.6/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until
